In [ ]:
!pip install transformers

In [7]:
!unzip /content/commonlitreadabilityprize.zip

Archive:  /content/commonlitreadabilityprize.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [3]:
from transformers import DistilBertTokenizer, RobertaTokenizer
roberta = 'roberta-base'

# Defining DistilBERT tokonizer

# Defining RoBERTa tokinizer
tokenizer = RobertaTokenizer.from_pretrained(roberta, do_lower_case=True, padding=True)

In [8]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.columns

Index(['id', 'url_legal', 'license', 'excerpt', 'target', 'standard_error'], dtype='object')

In [20]:
text = list(train.iloc[:,3])
label = list(train.iloc[:,4])

In [10]:
inputs= tokenizer(list(text), max_length=500,padding=True)

In [19]:
input_ids = list(inputs['input_ids'])
mask = list(inputs['attention_mask'])

In [ ]:
from transformers import TFRobertaModel
import tensorflow as tf

transformer_model = TFRobertaModel.from_pretrained('roberta-base')


In [ ]:
input_ids = tf.keras.layers.Input(shape=(322,), name='input_token', dtype='int32')
input_masks_ids = tf.keras.layers.Input(shape=(322,), name='masked_token', dtype='int32')
x = transformer_model(input_ids, input_masks_ids)
x = x.last_hidden_state
x = tf.keras.layers.Dense(256)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(2048)(x)
x = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = x)

In [16]:
for layer in model.layers[:-4]:
  layer.trainable = False

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 322)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 322)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode TFBaseModelOutputWit 124645632   input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
dense (Dense)                   (None, 322, 256)     196864      tf_roberta_model[0][0]       

In [17]:
model.compile(optimizer= 'adam' , loss= tf.keras.losses.MSE)


In [21]:
dataset = tf.data.Dataset.from_tensor_slices(((input_ids, mask), label))

In [22]:
dataset = dataset.batch(8)

In [ ]:
model.fit(
    dataset,
    epochs=100)

Epoch 1/100
355/355 [==============================] - 184s 479ms/step - loss: 145804.4935 - accuracy: 0.0011
Epoch 2/100
355/355 [==============================] - 170s 479ms/step - loss: 1977.6633 - accuracy: 0.0000e+00
Epoch 3/100
355/355 [==============================] - 170s 479ms/step - loss: 496.2901 - accuracy: 0.0000e+00
Epoch 4/100
355/355 [==============================] - 170s 478ms/step - loss: 180.7397 - accuracy: 0.0000e+00
Epoch 5/100
355/355 [==============================] - 170s 478ms/step - loss: 93.1953 - accuracy: 0.0000e+00
Epoch 6/100
355/355 [==============================] - 170s 478ms/step - loss: 110.5994 - accuracy: 0.0000e+00
Epoch 7/100
355/355 [==============================] - 170s 479ms/step - loss: 294.6429 - accuracy: 0.0000e+00
Epoch 8/100
355/355 [==============================] - 171s 482ms/step - loss: 355.0546 - accuracy: 0.0000e+00
Epoch 9/100
355/355 [==============================] - 171s 481ms/step - loss: 43.4437 - accuracy: 0.0011
Epoch 1